In [1]:
import tensorflow as tf
from ASR_Network import ASR_Network
from DataPipe import DataPipeFactory
from util_function import inform_pooling

/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/Users/jackwang/.local/share/virtualenvs/Code-_CZGGnvj/lib/python3.9/site-packages/tensorflow_io/p

In [2]:
# Disable all GPUS
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [3]:
ds = DataPipeFactory('../DataFolder/Tensorflow_DataRecord/Student_Answer_Record.tfrecord',
                     '../DataFolder/Siri_Related/Siri_Reference_Sample',
                     '../DataFolder/Siri_Related/Siri_Dense_Index', cache='../DataFolder/cache/datapipe/cached')
# dsp = ds.get_batch_data(10)
# it = iter(dsp)
ds.get_raw_data()

<_PrefetchDataset element_spec={'stu_mfcc': TensorSpec(shape=(None, 80), dtype=tf.float32, name=None), 'ref_mfcc': TensorSpec(shape=(4, None, 80), dtype=tf.float32, name=None), 'valid_stu_start': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'valid_stu_duration': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'valid_ref_word': TensorSpec(shape=(4, None), dtype=tf.int64, name=None), 'valid_ref_start': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'valid_ref_duration': TensorSpec(shape=(4, None), dtype=tf.float32, name=None), 'RecordName': TensorSpec(shape=(), dtype=tf.string, name=None), 'passage_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'MatchSegment': TensorSpec(shape=(4, None), dtype=tf.int64, name=None), 'MatchReference': TensorSpec(shape=(4, None), dtype=tf.int64, name=None)}>

In [4]:
base_feature = ([None, 80], 128, [128, 256, 384, 384], 3, 2)
base_feature_name = ('input_shape', 'feature_depth', 'channels_list', 'filter_size', 'stack_size')
dense_feature = ([896], 256, [256, 256], 1)
dense_feature_name = ('input_shape', 'output_shape', 'channels_list', 'stack_size')
word_prediction = ([256], 1800, [256, 256], 1)
word_prediction_name = ('input_shape', 'output_shape', 'channels_list', 'stack_size')
# Generate dict using zip
# base_feature = dict(zip(base_feature_name, base_feature))
# dense_feature = dict(zip(dense_feature_name, dense_feature))
# word_prediction = dict(zip(word_prediction_name, word_prediction))
base_ratio = 125
hyper_para = {'base_feature': dict(zip(base_feature_name, base_feature)),
              'dense_feature': dict(zip(dense_feature_name, dense_feature)),
              'word_prediction': dict(zip(word_prediction_name, word_prediction)),
              'base_ratio': base_ratio,
              'batch_num': 10,
              'margin': 0.4}
C = ASR_Network(**hyper_para)

In [5]:
def unpack(d):
    value_s = d['stu_mfcc']
    start_s = tf.RaggedTensor.from_tensor(d['valid_stu_start'], padding=-1.)
    duration_s = tf.RaggedTensor.from_tensor(d['valid_stu_duration'], padding=-1.)

    #unpack with another key ref_mfcc, valid_ref_start, valid_ref_duration
    value_f = d['ref_mfcc']
    start_f = tf.RaggedTensor.from_tensor(d['valid_ref_start'], padding=-1.)
    duration_f = tf.RaggedTensor.from_tensor(d['valid_ref_duration'], padding=-1.)

    # unpack valid_ref_word

    words = tf.RaggedTensor.from_tensor(d['valid_ref_word'], padding=-1)
    return ((value_s, (start_s, duration_s)), (value_f, (start_f, duration_f))), words

In [6]:
dst = ds.get_batch_data(2, addition_map=unpack)

In [7]:
dst

<_PrefetchDataset element_spec=(((TensorSpec(shape=(2, None, 80), dtype=tf.float32, name=None), (RaggedTensorSpec(TensorShape([2, None]), tf.float32, 1, tf.int64), RaggedTensorSpec(TensorShape([2, None]), tf.float32, 1, tf.int64))), (TensorSpec(shape=(2, None, 80), dtype=tf.float32, name=None), (RaggedTensorSpec(TensorShape([2, None]), tf.float32, 1, tf.int64), RaggedTensorSpec(TensorShape([2, None]), tf.float32, 1, tf.int64)))), RaggedTensorSpec(TensorShape([2, None]), tf.int64, 1, tf.int64))>

In [8]:
it = iter(dst)

2023-04-20 02:14:57.485151: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [9]:
data = next(it)

In [10]:
def call(self, inputs, training=False, mask=None):
    audio, (start, duration) = inputs
    # compute the base network
    base_output, maps = self.base_network(audio, training=training)
    # combine base output and maps
    total_maps = [base_output] + maps
    # pooling the total maps
    pooled_maps = self.pooling(total_maps, start, duration)
    # compute the deep feature
    deep_feature = self.deep_feature(pooled_maps, training=training)
    # compute the word prediction
    word_prediction = self.word_prediction(deep_feature, training=training)
    return word_prediction, deep_feature

In [11]:
C(data[0][0])

(<tf.RaggedTensor [[[0.61900806, -0.6924678, 0.3453373, ..., 0.5345688, -0.7992587,
    0.060125336],
   [0.15832646, -0.7718807, 0.16642614, ..., -0.13603045, -1.0601312,
    0.58756655],
   [0.9603169, -0.20984064, 0.8390224, ..., 0.3225174, -0.81183946,
    0.010617867],
   ...,
   [1.5812459, 0.02937457, -0.03883384, ..., 0.6773226, -0.1859645,
    -1.1265874],
   [1.4834126, -0.057592627, -0.26173988, ..., 1.0564874, -0.284454,
    -1.1055346],
   [0.9493375, -0.8911254, -0.15515706, ..., 0.51474285, -0.4014134,
    -1.22148]]                                                       ,
  [[0.14365986, -0.6631809, 0.12682083, ..., -0.4728651, -0.77805454,
    -0.6189637],
   [0.63632345, -0.6835557, -1.4192319, ..., -0.94842374, -1.4261521,
    -1.3134607],
   [0.91634893, -2.0015836, -0.74420536, ..., -1.1273637, -1.4427704,
    -1.479621],
   ...,
   [0.03956808, -0.4102338, -0.98836976, ..., 0.44311556, -0.42057562,
    0.42977864],
   [-0.55339026, -0.11820942, -0.9104495, ..., 0.6

In [12]:
C.auto_balancing_layer.trainable_weights

[<tf.Variable 'auto_balancing:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [ ]:
C.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
C.fit(dst, epochs=1)

In [ ]:
nl = tf.keras.layers.BatchNormalization()

In [ ]:
Ragg1 = tf.RaggedTensor.from_row_splits(tf.random.normal([100,80]), row_splits=tf.range(0, 101, 5))
Ragg2 = tf.RaggedTensor.from_row_splits(tf.random.normal([100,80]), row_splits=tf.range(0, 101, 10))

In [ ]:
tf.ragged.map_flat_values(nl, Ragg2)

In [ ]:
class RaggedBatchNormalization(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.batch_norm = tf.keras.layers.BatchNormalization(**kwargs)

    def call(self, inputs, training=None, mask=None):
        # Pass `training` and `mask` arguments to the `call` method of the BatchNormalization layer
        return tf.ragged.map_flat_values(
            lambda x: self.batch_norm(x, training=training, mask=mask), inputs
        )

    def get_config(self):
        return super().get_config()

    @classmethod
    def from_config(cls, config):
        return super().from_config(config)

In [ ]:
rnl = RaggedBatchNormalization()

In [ ]:
rnl(Ragg1)

In [ ]:
nl(tf.random.normal([100,80]))

In [ ]:
Ragg1.ragged_rank

In [ ]:
Ragg1.flat_values

In [ ]:
data

In [ ]:
dst

In [ ]:
dst = ds.get_batch_data(10)

In [ ]:
dst

In [ ]:
dst, dse = ds.get_batch_data_with_eval(10, 4096, addition_map=unpack)

In [ ]:
dst

In [ ]:
data

In [14]:
C.deep_feature.output.shape

TensorShape([None, 256])